<a href="https://colab.research.google.com/github/allydrzewo/CS370-Assignments/blob/main/assignment-2/Assignment2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install anomalib qdrant-client torch torchvision matplotlib
!pip install lightning
!pip install kornia

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.4/899.4 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.7 MB/s eta 0:00:00


In [5]:
from anomalib.data.mvtec import MVTec
from anomalib.config import get_configurable_parameters

categories = ["tile", "leather", "grid"]
mvtec_dataset = MVTec(root="./mvtec_ad", category=categories)

from anomalib.models import Patchcore, EfficientAd
from anomalib.utils.metrics import compute_auroc

# PatchCore model setup
config = get_configurable_parameters(model_name="patchcore", dataset_path="./mvtec_ad", category=categories)
patchcore_model = Patchcore(config)

# Evaluate the model and compute AUROC
results_patchcore = patchcore_model.evaluate(mvtec_dataset)
auroc_patchcore = compute_auroc(results_patchcore)
print("PatchCore AUROC per category:", auroc_patchcore)
print("Average AUROC:", sum(auroc_patchcore.values()) / len(auroc_patchcore))

# EfficientAD model setup
config = get_configurable_parameters(model_name="efficientad", dataset_path="./mvtec_ad", category=categories)
efficientad_model = EfficientAd(config)

# Evaluate the model and compute AUROC
results_efficientad = efficientad_model.evaluate(mvtec_dataset)
auroc_efficientad = compute_auroc(results_efficientad)
print("EfficientAD AUROC per category:", auroc_efficientad)
print("Average AUROC:", sum(auroc_efficientad.values()) / len(auroc_efficientad))

ModuleNotFoundError: No module named 'anomalib.utils.data'

In [6]:
from qdrant_client import QdrantClient
import numpy as np

# Initialize Qdrant client
client = QdrantClient(":memory:")

# Extract features for PatchCore
def extract_features(model, dataset):
    features = {}
    for img, _ in dataset:
        feature = model.extract_features(img)
        features[img] = feature.cpu().numpy()
    return features

# Store features
def store_features_in_qdrant(features):
    for idx, (img, feature) in enumerate(features.items()):
        client.upsert("anomaly_detection", point_id=idx, payload={}, vector=feature)

patchcore_features = extract_features(patchcore_model, mvtec_dataset)
store_features_in_qdrant(patchcore_features)

# Query for similar images
def get_similar_images(query_img, top_k=5):
    query_feature = patchcore_model.extract_features(query_img).cpu().numpy()
    results = client.search("anomaly_detection", vector=query_feature, limit=top_k)
    return results

# Test with image
query_img = mvtec_dataset[0][0]
similar_images = get_similar_images(query_img)
print("Top 5 similar images:", similar_images)

NameError: name 'patchcore_model' is not defined

In [7]:
import matplotlib.pyplot as plt

def visualize_results(images, title):
    plt.figure(figsize=(15, 10))
    for i, img in enumerate(images):
        plt.subplot(1, len(images), i + 1)
        plt.imshow(img)
        plt.title(f"{title} {i+1}")
    plt.show()

visualize_results([mvtec_dataset[0][0], *similar_images], "Similar Images to Query")

NameError: name 'mvtec_dataset' is not defined